In [2]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [16]:
df = pd.read_csv("../titanic.csv")

In [20]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
81,973,0,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S
283,1175,1,3,"Touma, Miss. Maria Youssef",female,9.0,1,1,2650,15.2458,NaN,C
241,1133,1,2,"Christy, Mrs. (Alice Frances)",female,45.0,0,2,237789,30.0000,NaN,S
348,1240,0,2,"Giles, Mr. Ralph",male,24.0,0,0,248726,13.5000,NaN,S
242,1134,0,1,"Spedden, Mr. Frederic Oakley",male,45.0,1,1,16966,134.5000,E34,C


In [22]:
#let's starts

In [24]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [26]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [28]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [30]:
# column Tranformer

In [32]:
#imputation Transform

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [58]:
# one Hot Encoding

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [60]:
# Scaling

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [62]:
# feature Selection

trf4 = SelectKBest(score_func=chi2,k=5)

In [64]:
#train the model

trf5 = DecisionTreeClassifier()

In [66]:
# CREATE PIPELINE

In [68]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [70]:
# alternat syntax fo creatig pipeline

#pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [72]:
#train

pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x0000027757C3EF20>)),
                ('trf5', DecisionTreeClassifier())])

In [74]:
# pridict thefr model

y_pred = pipe.predict(x_test)

In [76]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.5952380952380952

In [78]:
#Exporting the Pipeline

In [84]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))